In [8]:
# import문 보고 필요한 pip들 수동으로 설치하기 (requirements 하면 오류 많이 날수도)
# torch 최신버전이면 다 될듯!
!pwd

!mkdir checkpoint
!mkdir data
# 폴더 만들어졌으면 구글 드라이브에서 다운로드 
# model-v3 사용하면 됨 (임시로 epoch30으로 학습된 것)
# 구글 드라이브 url : 

# checkpoint 폴더 -> 모델 저장 (model-v3)
# data 폴더 -> txt 파일들 저장(category, answer_v2 / input_v2는 학습할 때 사용해서 다운 안받아도 됨)

/workspace/kobert-wellness-chatbot


In [1]:
import torch
import random
import os
import logging
from model.classifier import KoBERTforSequenceClassfication
from kobert_transformers import get_tokenizer
from model.dataloader import WellnessTextClassificationDataset

# warning 출력 안되게
logging.getLogger("transformers").setLevel(logging.ERROR)

In [2]:
# data 경로 지정
root_path = "."
category_path = f"{root_path}/data/category.txt"
answer_path = f"{root_path}/data/answer_v2.txt"
checkpoint_path = f"{root_path}/checkpoint/model-v3.pth"

In [3]:
def load_wellness_answer():

    c_f = open(category_path, 'r')
    a_f = open(answer_path, 'r')

    category_lines = c_f.readlines()
    answer_lines = a_f.readlines()

    category = {}
    answer = {}
    for line_num, line_data in enumerate(category_lines):
        data = line_data.split('    ')
        if len(data) != 2:
            print(f"Error in category file at line {line_num}: {line_data}")
        category[data[1][:-1]] = data[0]

    for line_num, line_data in enumerate(answer_lines):
        data = line_data.split('    ')
        keys = answer.keys()
        if len(data) != 2:
            print(f"Error in answer file at line {line_num}: {line_data}")
        if (data[0] in keys):
            answer[data[0]] += [data[1][:-1]]
        else:
            answer[data[0]] = [data[1][:-1]]

    return category, answer


def load_model(checkpoint_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = KoBERTforSequenceClassfication()
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    tokenizer = get_tokenizer()
    return model, tokenizer, device

def preprocess_input(tokenizer, sent, device, max_seq_len=512):
    index_of_words = tokenizer.encode(sent)
    token_type_ids = [0] * len(index_of_words)
    attention_mask = [1] * len(index_of_words)
    padding_length = max_seq_len - len(index_of_words)
    index_of_words += [0] * padding_length
    token_type_ids += [0] * padding_length
    attention_mask += [0] * padding_length
    data = {
        'input_ids': torch.tensor([index_of_words]).to(device),
        'token_type_ids': torch.tensor([token_type_ids]).to(device),
        'attention_mask': torch.tensor([attention_mask]).to(device),
    }
    return data

In [4]:
def get_answer(category, answer, output):
    softmax_logit = torch.softmax(output[0], dim=-1).squeeze()
    max_index = torch.argmax(softmax_logit).item()
    max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()
    if str(max_index) in category:
        answer_category = category[str(max_index)]
        answer_list = answer[category[str(max_index)]]
        answer_len = len(answer_list) - 1
        
        # 랜덤으로 답변을 불러와서 수정 필요....
        answer_index = random.randint(0, answer_len)
        return answer_list[answer_index], answer_category, max_index_value
    else:
        return "키가 딕셔너리에 존재하지 않습니다.", None, max_index_value

def chatbot_main(checkpoint_path, category, answer):
    model, tokenizer, device = load_model(checkpoint_path)
    while True:
        # 터미널에서 한글 입력할 때 띄어쓰기나 백스페이스 누르면 문자열 못읽고 종료되는 경우 있음
        # 수정할 예정
        sent1 = input('\nQuestion: ')
        sent = str(sent1)
        if '종료' in sent:
            break
        if '안녕?' in sent or '안녕!' in sent or '안녕' in sent:
            print('Answer : 반가워요! 저는 기룡이에요!')
            continue
        data = preprocess_input(tokenizer, sent, device, 512)
        output = model(**data)
        answer, category, max_index_value = get_answer(category, answer, output)
        print(f'Answer: {answer} \nindex: {category},{max_index_value} \nsoftmax_value: {max_index_value}')
        print('-' * 70)

In [5]:
category, answer = load_wellness_answer()

In [ ]:
# 챗봇 실행
chatbot_main(checkpoint_path, category, answer)